In [56]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [57]:
CLIENT_ID = '12M3TVS2JUKRUCXV2GT5AOJQKV2XHNFNCGR0ZKQAPG5GIVYG' # your Foursquare ID
CLIENT_SECRET = 'HK5NFVM2D0CINJAU4CMMICEJXBUSJQXTG1TFM25ULPSSSJ35' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 12M3TVS2JUKRUCXV2GT5AOJQKV2XHNFNCGR0ZKQAPG5GIVYG
CLIENT_SECRET:HK5NFVM2D0CINJAU4CMMICEJXBUSJQXTG1TFM25ULPSSSJ35


In [58]:
#Geting latitude and Longitude from Downtown Rio de Janeiro
neighborhood_latitude = -22.907 # neighborhood latitude value
neighborhood_longitude = -43.1819 # neighborhood longitude value

neighborhood_name = 'Downtown Rio' # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Downtown Rio are -22.907, -43.1819.


In [59]:
#top 100 venues that are in Downtown Rio within a radius of 500 meters.
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=12M3TVS2JUKRUCXV2GT5AOJQKV2XHNFNCGR0ZKQAPG5GIVYG&client_secret=HK5NFVM2D0CINJAU4CMMICEJXBUSJQXTG1TFM25ULPSSSJ35&v=20180605&ll=-22.907,-43.1819&radius=500&limit=100'

In [60]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e5812be211536001bacb13a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Centro',
  'headerFullLocation': 'Centro, Rio de Janeiro',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 100,
  'suggestedBounds': {'ne': {'lat': -22.902499995499994,
    'lng': -43.17702385422383},
   'sw': {'lat': -22.911500004500006, 'lng': -43.186776145776165}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e6146e82271573ad71b86d1',
       'name': 'Dança CCC',
       'location': {'address': 'Centro Cultural Carioca',
        'crossStreet': '3º Andar',
        'lat': -22.906403558576535,
        'lng': -43.181832344578694,
        'labeledLatLngs

In [61]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
#clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(30)

,name,categories,lat,lng
0,Dança CCC,Dance Studio,-22.906404,-43.181832
1,Rio Scenarium,Music Venue,-22.908255,-43.183938
2,Centro de Arte Hélio Oiticica,Art Gallery,-22.906106,-43.183425
3,CRAB - Centro Sebrae de Referência do Artesana...,Cultural Center,-22.907662,-43.183819
4,Casa do Choro,Music Venue,-22.906581,-43.180588
5,Adega do Pimenta,German Restaurant,-22.906858,-43.181992
6,Espaço Franklin,Music Venue,-22.905169,-43.182909
7,Lilia Restaurante,Brazilian Restaurant,-22.909135,-43.184018
8,Casa Cavé,Café,-22.905825,-43.178962
9,Espaço Cultural BNDES,Cultural Center,-22.908349,-43.179394


In [62]:
nearby_venues.shape

(100, 4)

In [63]:
nearby_venues['categories']

0                      Dance Studio
1                       Music Venue
2                       Art Gallery
3                   Cultural Center
4                       Music Venue
5                 German Restaurant
6                       Music Venue
7              Brazilian Restaurant
8                              Café
9                   Cultural Center
10                      Pastry Shop
11                      Coffee Shop
12                          Theater
13                       Pastelaria
14    Paper / Office Supplies Store
15                      Art Gallery
16                              Bar
17                         Dive Bar
18             Brazilian Restaurant
19                       Steakhouse
20                      Flea Market
21                       Smoke Shop
22              Japanese Restaurant
23                   Sandwich Place
24                   Clothing Store
25                              Gym
26                         Tea Room
27                      Past

In [65]:
categories_data = nearby_venues.groupby('categories').count()
categories_data.head()

,name,lat,lng
categories,,,
Art Gallery,2,2,2
Bakery,3,3,3
Bar,3,3,3
Bed & Breakfast,1,1,1
Beer Store,1,1,1


In [72]:
#Clusterinh the venues' categories
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(categories_data)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 2], dtype=int32)

In [75]:
# add clustering labels
categories_data.head(30)

,Cluster Labels,name,lat,lng
categories,,,,
Art Gallery,0,2,2,2
Bakery,0,3,3,3
Bar,0,3,3,3
Bed & Breakfast,0,1,1,1
Beer Store,0,1,1,1
Bistro,0,1,1,1
Brazilian Restaurant,1,15,15,15
Buffet,0,1,1,1
Burger Joint,0,2,2,2


In [76]:
categories_data.reset_index()

,categories,Cluster Labels,name,lat,lng
0,Art Gallery,0,2,2,2
1,Bakery,0,3,3,3
2,Bar,0,3,3,3
3,Bed & Breakfast,0,1,1,1
4,Beer Store,0,1,1,1
5,Bistro,0,1,1,1
6,Brazilian Restaurant,1,15,15,15
7,Buffet,0,1,1,1
8,Burger Joint,0,2,2,2
9,Café,2,5,5,5


In [77]:
#Insert cluster Labels into nearby_venues
nearby_venues_merged = nearby_venues.merge(categories_data, on='categories', how='left')
nearby_venues_merged.head()

,name_x,categories,lat_x,lng_x,Cluster Labels,name_y,lat_y,lng_y
0,Dança CCC,Dance Studio,-22.906404,-43.181832,0,2,2,2
1,Rio Scenarium,Music Venue,-22.908255,-43.183938,0,3,3,3
2,Centro de Arte Hélio Oiticica,Art Gallery,-22.906106,-43.183425,0,2,2,2
3,CRAB - Centro Sebrae de Referência do Artesana...,Cultural Center,-22.907662,-43.183819,0,2,2,2
4,Casa do Choro,Music Venue,-22.906581,-43.180588,0,3,3,3


In [78]:
#drop and rename columns

dropcolumns = nearby_venues_merged.drop(['name_y', 'lat_y', 'lng_y'], axis = 1)
dropcolumns.head()

,name_x,categories,lat_x,lng_x,Cluster Labels
0,Dança CCC,Dance Studio,-22.906404,-43.181832,0
1,Rio Scenarium,Music Venue,-22.908255,-43.183938,0
2,Centro de Arte Hélio Oiticica,Art Gallery,-22.906106,-43.183425,0
3,CRAB - Centro Sebrae de Referência do Artesana...,Cultural Center,-22.907662,-43.183819,0
4,Casa do Choro,Music Venue,-22.906581,-43.180588,0


In [79]:
downtown_rio_venues = dropcolumns.rename(columns={"name_x": "name", "categories": "categories", "lat_x":"lat", "lng_x": "lng", "Cluster Labels": "Cluster Labels" })
downtown_rio_venues

,name,categories,lat,lng,Cluster Labels
0,Dança CCC,Dance Studio,-22.906404,-43.181832,0
1,Rio Scenarium,Music Venue,-22.908255,-43.183938,0
2,Centro de Arte Hélio Oiticica,Art Gallery,-22.906106,-43.183425,0
3,CRAB - Centro Sebrae de Referência do Artesana...,Cultural Center,-22.907662,-43.183819,0
4,Casa do Choro,Music Venue,-22.906581,-43.180588,0
5,Adega do Pimenta,German Restaurant,-22.906858,-43.181992,0
6,Espaço Franklin,Music Venue,-22.905169,-43.182909,0
7,Lilia Restaurante,Brazilian Restaurant,-22.909135,-43.184018,1
8,Casa Cavé,Café,-22.905825,-43.178962,2
9,Espaço Cultural BNDES,Cultural Center,-22.908349,-43.179394,0


In [80]:
# create map downtown rio
latitude = -22.907 
longitude = -43.1819
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_rio_venues['lat'], downtown_rio_venues['lng'], downtown_rio_venues['name'], downtown_rio_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [81]:
#Cluster 1
downtown_rio_venues.loc[downtown_rio_venues['Cluster Labels'] == 0, downtown_rio_venues.columns[[1] + list(range(5, downtown_rio_venues.shape[1]))]]

,categories
0,Dance Studio
1,Music Venue
2,Art Gallery
3,Cultural Center
4,Music Venue
5,German Restaurant
6,Music Venue
9,Cultural Center
10,Pastry Shop
11,Coffee Shop


In [86]:
#Cluster 2
downtown_rio_venues.loc[downtown_rio_venues['Cluster Labels'] == 1, downtown_rio_venues.columns[[1] + list(range(5, downtown_rio_venues.shape[1]))]]

,categories
7,Brazilian Restaurant
18,Brazilian Restaurant
28,Brazilian Restaurant
50,Brazilian Restaurant
52,Brazilian Restaurant
61,Brazilian Restaurant
64,Brazilian Restaurant
67,Brazilian Restaurant
69,Brazilian Restaurant
74,Brazilian Restaurant


In [87]:
#Cluster 3
downtown_rio_venues.loc[downtown_rio_venues['Cluster Labels'] == 2, downtown_rio_venues.columns[[1] + list(range(5, downtown_rio_venues.shape[1]))]]

,categories
8,Café
32,Middle Eastern Restaurant
40,Café
44,Middle Eastern Restaurant
46,Middle Eastern Restaurant
54,Middle Eastern Restaurant
55,Middle Eastern Restaurant
70,Café
78,Middle Eastern Restaurant
79,Middle Eastern Restaurant
